## 1. Impprt Dependacies

In [42]:
import os
import numpy as np
import random
from gymnasium import Env
from gymnasium.spaces import Box, Discrete, MultiDiscrete, Dict
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

## 3. Building an Environment
- Build and agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees

In [44]:
def current_density_stdev():
    return np.random.uniform(0, 3) 

In [1]:
import numpy as np
import random
from gymnasium import Env
from gymnasium.spaces import Box

In [45]:
   self.action_space = Tuple((Discrete(7, start=1),Box(low=5, high=40, shape=(4,), dtype=int)))
        self.observation_space = Tuple((Discrete(7, start=1),Box(low=0, high=np.inf, shape=(8,), dtype=np.float32)))
        self.min_density = current_density_stdev()

IndentationError: unexpected indent (2144672322.py, line 2)

In [74]:
Dict({'com':Discrete(7, start=1), 'side-timers':Box(low=5, high=40, shape=(4,), dtype=int)}).sample()

OrderedDict([('com', 5), ('side-timers', array([38, 17, 23, 38]))])

In [76]:
Dict({'com':Discrete(7, start=1), 'side-timers':Box(low=0, high=np.inf, shape=(8,), dtype=np.float32)}).sample()

OrderedDict([('com', 6),
             ('side-timers',
              array([0.18967883, 0.40962645, 1.8089591 , 1.1234088 , 0.316804  ,
                     0.29596582, 0.56445277, 0.0802756 ], dtype=float32))])

In [2]:
class TrafficSimulation(Env):
    def __init__(self):
        super(TrafficSimulation, self).__init__()
        
        # Define the observation space as a Dict
        self.observation_space = Dict({
            "combination": Discrete(7),  # Current route combination (com1 to com7)
            "densities": Box(low=0, high=np.inf, shape=(8,), dtype=np.float32)  # Densities of S1 to S8
        })
        
        # Define the action space as a MultiDiscrete
        self.action_space = MultiDiscrete([
            7,  # com: 7 possible combinations [1, 7]
            36, 36, 36, 36  # side-timers: 4 values each ranging from 5 to 40 (5 + i where i is in range(0, 36))
        ])
        
        # Initialize state
        self.min_density = current_density_stdev()
        self.state = self.min_density
        self.max_steps = 100  # Limit to prevent infinite loops
        self.current_step = 0

    def step(self, action):
        """
        Take an action in the environment.
        """
        # Simulate new density with some conditions
        new_density = current_density_stdev()

        # Calculate reward
        if self.min_density > new_density:
            reward = 1
            self.min_density = new_density
        elif self.min_density == new_density:
            reward = 0
        else:
            reward = -1

        # Check if the episode is done
        self.current_step += 1
        done = self.current_step >= self.max_steps  # Ensure the episode ends after max_steps
        
        # Optional additional info
        info = {}

        # Update state
        self.state = new_density
        
        return np.array([self.state], dtype=np.float32), reward, done, info

    def reset(self, seed=None, options=None):
        """
        Reset the environment state.
        """
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
        
        # Reset state to initial or random density
        self.state = current_density_stdev()
        self.min_density = self.state
        self.current_step = 0  # Reset step count
        
        # Print statement can be uncommented for debugging
        # print("Environment reset: Initial Density:", self.state)
        
        return np.array([self.state], dtype=np.float32), {}
        
    def render(self):
        # Implement visualization if needed
        pass

def current_density_stdev():
    """
    Dummy function to return a simulated density value.
    Modified to sometimes return lower densities.
    """
    # Randomly decide if we want to generate a higher or lower density
    if random.random() > 0.7:  # 30% chance to decrease density
        return max(0, np.random.uniform(0, 5))  # Lower range for new densities
    else:
        return np.random.uniform(0, 10)  # Original range


In [3]:
env = TrafficSimulation()

NameError: name 'Dict' is not defined

In [87]:
env.observation_space.sample()

OrderedDict([('combination', 6),
             ('densities',
              array([1.9184334 , 0.9233296 , 0.29225418, 0.9787818 , 0.06363887,
                     0.82842344, 0.3184587 , 0.16957685], dtype=float32))])

In [67]:
env.action_space.sample()

array([25, 37,  8, 29])

## 4. Test Environment

In [68]:
action = env.action_space.sample()

In [69]:
env.reset()
env.step(action)

(array([6.7581024], dtype=float32), -1, False, {})

In [80]:
# Initialize the environment
env = TrafficSimulation()

# Number of episodes to run
episodes = 5
for episode in range(1, episodes + 1):
    # Reset the environment
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        
        # Sample a random action from the action space
        action = env.action_space.sample()
        
        # Take the action in the environment
        obs, reward, done, info = env.step(action)
        
        # Accumulate the reward
        score += reward

    print('Episode: {} Score: {}'.format(episode, score))

env.close()

Episode: 1 Score: -96
Episode: 2 Score: -90
Episode: 3 Score: -92
Episode: 4 Score: -86
Episode: 5 Score: -88


## 5. Train Model

In [81]:
from stable_baselines3 import DQN

In [84]:
log_path = os.path.join('Training' , 'Logs')
model = PPO("MultiInputPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.box.Box'>, <class 'gymnasium.spaces.discrete.Discrete'>, <class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>, <class 'gymnasium.spaces.multi_binary.MultiBinary'>) as action spaces but Dict('com': Discrete(7, start=1), 'side-timers': Box(5, 40, (4,), int64)) was provided

In [43]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -14.4    |
| time/              |          |
|    fps             | 626      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -8.32        |
| time/                   |              |
|    fps                  | 430          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0025594546 |
|    clip_fraction        | 0.0208       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.417       |
|    explained_variance   |

## 6. Save Model

In [49]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [50]:
model.save(shower_path)

## Evaluvation

In [52]:
del model

In [54]:
model = PPO.load(shower_path)

In [58]:
evaluate_policy(model, env, n_eval_episodes=10)

(12.0, 58.787753826796276)